In [795]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, OneHotEncoder, StandardScaler
from sklearn.linear_model import SGDRegressor
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.compose import TransformedTargetRegressor
from scipy.stats import boxcox
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import VarianceThreshold
import ml_utils

# Assignment 2 - Regression
### Predict the TOTAL COMPENSATION for this year. 

The data file provided is a salary survey for tech workers in Europe. We want to predict the total amount of compensation they bring in each year, based off of the details of their work. 

Some notes that will be important:
<ul>
    <li>The total compensation will need to be constructed, there is a column for salary, "Yearly brutto salary (without bonus and stocks) in EUR", as well as a column for bonus compensation, "Yearly bonus + stocks in EUR". 
    <li>Some categorical variables will need some work, and there isn't generally an exact answer. The main concern is things with categories that have a bunch of values with a very small count. For example, if there is only 1 person in City X, then that value likely needs to be addressed. We don't want it encoded into a new column of one 1 and thousands of 0s. 
    <li>There is an article exploring some of the data here: https://www.asdcode.de/2021/01/it-salary-survey-december-2020.html
    <li>Imputation and a bit of data manipulation will be required. 
    <li>Use any regression method you'd like. Some ones are closely related to what we've done, you may want to look at them, e.g. ExtraTreesRegressor. 
    <li>Initial accurracy, and potentially final accuracy, may not be great. When I made a plain model will little optimization the errors were large and the R2 was low. There is lots of room for optimization. 
    <li>Research challenge - try some work on the target, look into TransformedTargetRegressor and see if that helps. Recall in stats when we had skewed distributions... Maybe it helps, maybe it doesn't. 
    <li>EDA and data prep are up to you - you'll probably need to do a little exploring to figure out what cleanup is needed. When I did it, I did things kind of iteratively when I did it. For example, look at the value counts, figure out how to treat the different categories, clean something up, look at the results, potentially repeat if needed. After you figure out what needs to be done, you may be able to take some of those steps and incorporate them into a pipeline to be cleaner....
    <li><b>CRITICAL - Please make sure your code runs with RUN ALL. It should load the data that you're given, do all the processing, and spit out results. Comment out or remove anything that you've cleaned up and don't need - e.g. if you scaled a value manually, then moved that into a pipeline, don't leave the original scaling code active when the file is run.</b>
</ul>

### Details and Deliverables

You'll need to build code to produce the predictions. In particular, there's a few things that'll be marked:
<ul>
    <li>Please add a markdown cell at the bottom, and put in a few notes addressing the following:
    <ul>
        <li> Accuracy of your models with/without feature selection. Include both train/test for each. Please use R2 and RMSE. 
        <li> Feature Selection - Please identify what you did for feature selection. No need for a long explaination, something along the lines of "I did X, and the result was that 4 features were removed". Try at least 2 things. 
        <li> Hyperparameter Changes / Grid Search Improvements. What did you try, and why. Similar explaination to above, short. 
        <li> Overall this section should be roughly as long as this intro block - just outline what the results were, what you did to improve, and the results after. 
        <li> If you could use titles/bullet points I'd really appreciate it. 
    </ul>
    <li>Grade Breakdown:
    <ul>
        <li> Code is readable, there are comments: 20%
        <li> Explaination as defined above: 60% (20% each point)
        <li> Accuracy: 20% As compared to everyone else. This will be generously graded, I won't be surprised if overall accuracy is low for most people. 
    </ul>
</ul>

In [796]:
#Load Data
df = pd.read_csv("data/Euro_Salary.csv")
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Timestamp,1253,1248,24/11/2020 13:55:19,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,1226.0,NaN,NaN,NaN,32.509788,5.663804,20.0,29.0,32.0,35.0,69.0
Gender,1243,3,Male,1049,NaN,NaN,NaN,NaN,NaN,NaN,NaN
City,1253,119,Berlin,681,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Position,1247,148,Software Engineer,387,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total years of experience,1237,48,10,138,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Years of experience in Germany,1221,53,2,195,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Seniority level,1241,24,Senior,565,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Your main technology / programming language,1126,256,Java,184,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Other technologies/programming languages you use often,1096,562,Javascript / Typescript,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [797]:
df.sample(10)

,Timestamp,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Yearly brutto salary (without bonus and stocks) in EUR,Yearly bonus + stocks in EUR,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type
126,24/11/2020 12:00:59,31.0,Male,Berlin,Software Engineer,9,1,Senior,"C#, .net core","Python, Javascript / Typescript, .NET, AWS, Az...",60000.0,6000,25,Full-time employee,Unlimited contract,English,11-50,Startup
919,30/11/2020 09:57:43,32.0,Male,Munich,Frontend Developer,10,4,Lead,js,Javascript / Typescript,80000.0,85000,25,Full-time employee,Unlimited contract,English,11-50,Startup
99,24/11/2020 11:50:52,30.0,Male,Berlin,Backend Developer,8,2.5,Senior,Scala,"Python, Kotlin, Java / Scala, SQL, Kubernetes,...",80000.0,NaN,25,Full-time employee,Unlimited contract,English,1000+,Product
593,25/11/2020 13:58:38,30.0,Male,Berlin,Data Scientist,6,3,Senior,NaN,"Python, SQL, AWS, Google Cloud, Docker",72000.0,85000,27,Full-time employee,Unlimited contract,English,1000+,Startup
419,24/11/2020 20:26:58,28.0,Female,Berlin,Software Engineer,3,3,Middle,Javascript,"PHP, Javascript / Typescript",60000.0,0,28,Full-time employee,Unlimited contract,English,101-1000,Startup
970,30/11/2020 15:27:53,29.0,Male,Berlin,Backend Developer,9,6,Senior,Java,"Kotlin, Java / Scala, SQL, AWS",75000.0,NaN,25,Full-time employee,Unlimited contract,English,51-100,Startup
1112,07/12/2020 21:22:11,32.0,Male,Munich,Software Engineer,10,3,Lead,Swift,Java / Scala,75000.0,5000,27,Full-time employee,Unlimited contract,English,101-1000,Product
708,25/11/2020 22:18:59,NaN,Male,Frankfurt,Software Engineer,18,1,Senior,NaN,"Javascript / Typescript, .NET, SQL, Google Cloud",81000.0,3375,30,Full-time employee,Unlimited contract,English,1000+,Product
157,24/11/2020 12:16:38,38.0,Male,Cologne,Software Engineer,11,5,Senior,PHP,"PHP, Javascript / Typescript, Go, AWS, Google ...",55000.0,NaN,26,Full-time employee,Unlimited contract,English,101-1000,Product
546,25/11/2020 11:41:13,28.0,Male,Munich,Software Engineer,5,4,Middle,Java,"Javascript / Typescript, Java / Scala, AWS, Do...",56000.0,8000,25,Full-time employee,Unlimited contract,English,11-50,Startup


In [798]:
#df.info()
df["Other technologies/programming languages you use often"].value_counts()

Javascript / Typescript                                                                   44
Python                                                                                    37
SQL                                                                                       31
AWS, Docker                                                                               16
Kotlin                                                                                    15
                                                                                          ..
Python, Ruby, Java / Scala, Go, Rust, AWS, Kubernetes, Docker, Terraform, helm, packer     1
Kotlin, SQL, AWS, Kubernetes, Docker                                                       1
Python, R, Go                                                                              1
C/C++, PHP, SQL                                                                            1
Python, AWS, Kubernetes, Docker, terraform, ansible                   

In [799]:
df.head()

,Timestamp,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Yearly brutto salary (without bonus and stocks) in EUR,Yearly bonus + stocks in EUR,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type
0,24/11/2020 11:14:15,26.0,Male,Munich,Software Engineer,5,3,Senior,TypeScript,"Kotlin, Javascript / Typescript",80000.0,5000,30,Full-time employee,Unlimited contract,English,51-100,Product
1,24/11/2020 11:14:16,26.0,Male,Berlin,Backend Developer,7,4,Senior,Ruby,NaN,80000.0,NaN,28,Full-time employee,Unlimited contract,English,101-1000,Product
2,24/11/2020 11:14:21,29.0,Male,Berlin,Software Engineer,12,6,Lead,Javascript / Typescript,"Javascript / Typescript, Docker",120000.0,120000,30,Self-employed (freelancer),Temporary contract,English,101-1000,Product
3,24/11/2020 11:15:24,28.0,Male,Berlin,Frontend Developer,4,1,Junior,Javascript,NaN,54000.0,NaN,24,Full-time employee,Unlimited contract,English,51-100,Startup
4,24/11/2020 11:15:46,37.0,Male,Berlin,Backend Developer,17,6,Senior,C# .NET,".NET, SQL, AWS, Docker",62000.0,NaN,29,Full-time employee,Unlimited contract,English,101-1000,Product


##### Ran my eda and got a good look at the data and value counts

In [800]:
# df_eda = ml_utils.edaDF(df, "total compensation")
# print(df_eda.giveTarget())

In [801]:
# df_eda.fullEDA(k=1.5, scatterplot=True, optional_countplots=False, optional_histplots=False) 

##### Handled my missing values and replaced them with the median

In [802]:
# specify the columns that contain non-numeric values
columns = ['Total years of experience', 'Years of experience in Germany', 'Yearly bonus + stocks in EUR', 'Number of vacation days']

# convert the non-numeric values in the specified columns to NaN
df[columns] = df[columns].apply(pd.to_numeric, errors='coerce')

# replace the NaN values with the median
for col in columns:
    median = df[col].median()
    df[col].fillna(median, inplace=True)

# replace the commas with decimals
df = df.applymap(lambda x: x.replace(',', '.') if isinstance(x, str) else x)

df.sample(15)

,Timestamp,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Yearly brutto salary (without bonus and stocks) in EUR,Yearly bonus + stocks in EUR,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type
682,25/11/2020 19:15:16,32.0,Male,Berlin,Software Engineer,9.0,3.0,Lead,Embedded,Python. C/C++. Go,73000.0,1000.0,28.0,Full-time employee,Unlimited contract,English,101-1000,Product
408,24/11/2020 20:02:34,30.0,Male,Munich,Mobile Developer,8.0,2.0,Lead,Swift,Swift,75000.0,86000.0,24.0,Full-time employee,Unlimited contract,English,11-50,Startup
942,30/11/2020 12:08:57,37.0,Female,Berlin,Data Scientist,8.0,0.5,Middle,Python,SQL. AWS,67000.0,0.0,28.0,Full-time employee,Unlimited contract,English,1000+,Startup
82,24/11/2020 11:44:34,30.0,Male,Berlin,Software Engineer,10.0,2.0,Senior,Scala,Python. Java / Scala. AWS. Docker,72000.0,5000.0,28.0,Full-time employee,Unlimited contract,English,101-1000,Product
790,26/11/2020 20:41:46,33.0,Male,Berlin,Backend Developer,13.0,1.0,Senior,Java,Kotlin. Java / Scala. SQL. AWS. Google Cloud. ...,110000.0,5000.0,24.0,Full-time employee,Unlimited contract,English,101-1000,Product
524,25/11/2020 10:53:48,29.0,Female,Berlin,Software Developer in Test,9.0,4.0,Lead,Java,Javascript / Typescript. SQL. Docker,66000.0,0.0,25.0,Full-time employee,Unlimited contract,English,101-1000,Startup
614,25/11/2020 14:37:25,31.0,Male,Berlin,Software Engineer,9.0,5.0,Lead,javascript,AWS. Docker,99000.0,99000.0,30.0,Full-time employee,Unlimited contract,English,101-1000,Startup
1102,06/12/2020 21:17:41,32.0,Female,Berlin,Computational linguist,5.0,5.0,Middle,Java,Python. Docker,50400.0,0.0,29.0,Full-time employee,Unlimited contract,English,11-50,Product
693,25/11/2020 20:31:41,36.0,Male,Berlin,Product Manager,10.0,2.0,Senior,NaN,SQL,85000.0,5000.0,30.0,Full-time employee,Unlimited contract,English,1000+,Product
483,25/11/2020 08:08:50,28.0,Male,Berlin,Frontend Developer,2.0,2.0,Junior,JavaScript,Javascript / Typescript. Google Cloud,47000.0,5000.0,26.0,Full-time employee,Unlimited contract,German,1000+,Publishing and Technology


In [803]:
len(df)

1253

##### Created a column for total compensation to be used in my model

In [804]:
df["total compensation"] = df["Yearly brutto salary (without bonus and stocks) in EUR"] + df["Yearly bonus + stocks in EUR"]
df = df.drop(columns=["Yearly brutto salary (without bonus and stocks) in EUR", "Yearly bonus + stocks in EUR"])
df.sample(10)

,Timestamp,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type,total compensation
1026,01/12/2020 11:17:35,25.0,Male,Berlin,Software Engineer,6.0,0.0,Middle,Scala,Python. Javascript / Typescript. Java / Scala....,30.0,Full-time employee,Unlimited contract,English,1000+,Product,66000.0
452,24/11/2020 22:10:34,40.0,Female,Berlin,Fullstack Developer,8.0,8.0,Senior,C#,PHP. Javascript / Typescript. .NET. SQL. Azure...,26.0,Full-time employee,Unlimited contract,German,51-100,Product,58500.0
717,25/11/2020 23:42:47,35.0,Female,Munich,Backend Developer,4.0,4.0,Middle,PHP,PHP. Javascript / Typescript. Java / Scala,30.0,Full-time employee,Unlimited contract,German,101-1000,Insurance,55000.0
667,25/11/2020 18:31:48,34.0,Male,Munich,Software Engineer,12.0,8.0,Senior,Java,Java / Scala. SQL. Go. Google Cloud. Kubernete...,30.0,Full-time employee,Unlimited contract,English,101-1000,Product,74000.0
536,25/11/2020 11:18:27,26.0,Male,Berlin,Data Scientist,5.0,3.0,Middle,Python,SQL. AWS. Kubernetes. Docker,26.0,Full-time employee,Unlimited contract,English,101-1000,Startup,59000.0
131,24/11/2020 12:04:15,34.0,Male,Munich,NaN,17.0,6.0,Lead,Java,Python. Java / Scala. R. SQL. Go. AWS. Google ...,32.0,Full-time employee,Unlimited contract,German,1000+,Consulting / Agency,140000.0
1140,09/12/2020 13:37:29,26.0,Female,Berlin,Mobile Developer,8.0,3.0,Middle,c++,C/C++. SAP / ABAP,36.0,Self-employed (freelancer),Temporary contract,German,101-1000,Product,42000.0
32,24/11/2020 11:27:39,29.0,Female,Berlin,QA Engineer,7.0,1.0,Middle,NaN,Javascript / Typescript,26.0,Full-time employee,Unlimited contract,English,51-100,Startup,47000.0
815,27/11/2020 06:57:33,37.0,Male,Berlin,QA Engineer,8.0,7.0,Middle,NaN,NaN,30.0,Full-time employee,Unlimited contract,German,101-1000,Product,59000.0
887,28/11/2020 12:31:36,31.0,Male,Berlin,Backend Developer,9.0,9.0,Senior,Grails. Groovy,Java / Scala. SQL. AWS,28.0,Full-time employee,Unlimited contract,English,101-1000,Product,67473.0


##### Did a bit of my own feature selection to group unimportant values into an 'Other' category so it can be ran through my Onehotencoder better 

In [805]:
k = 5

# Loop through each column in the list
for col in ['City', 'Position ', 'Seniority level', 'Your main technology / programming language', 'Other technologies/programming languages you use often', 'Employment status', 'Сontract duration', 'Main language at work', 'Company type']:
    # Get the top k values for the column based on frequency
    top_k = df[col].value_counts().nlargest(k).index
    
    # Replace all values in the column that are not in the top k with the value "other"
    df[col] = np.where(df[col].isin(top_k), df[col], 'other')

##### Dropped timestamp as it was irrelevant. Then ran onehotencoder 

In [806]:
#Convert the Timestamp column into a pandas datetime object
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Extract relevant information from the datetime object
df['Year'] = df['Timestamp'].dt.year
df['Month'] = df['Timestamp'].dt.month
df['Day'] = df['Timestamp'].dt.day
df['Hour'] = df['Timestamp'].dt.hour

# Drop the original Timestamp column
df = df.drop('Timestamp', axis=1)

ohe = OneHotEncoder()

# Selecting categorical features for one hot encoding
categorical_features = ['Gender', 'City', 'Position ', 'Seniority level', 'Your main technology / programming language', 'Other technologies/programming languages you use often', 'Employment status', 'Сontract duration', 'Main language at work', 'Company type']

# Transforming categorical data into numerical data
ohe_features = ohe.fit_transform(df[categorical_features])

# Creating a dataframe from the transformed data
ohe_features_df = pd.DataFrame(ohe_features.toarray(), columns=ohe.get_feature_names(categorical_features))

# Adding the transformed features to the main dataframe
df = pd.concat([df, ohe_features_df], axis=1)

# Dropping the original categorical columns
df = df.drop(categorical_features, axis=1)

# Mapping the values of "Company size" to numerical data
df["Company size"] = df["Company size"].map({"up to 10": 1, "11-50": 2, "51-100": 3, "101-1000": 4, "1000+": 5})

# Viewing the first 10 rows of the dataframe
df.head(10)



c:\Users\Luke\anaconda3\envs\backup_env\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Age,Total years of experience,Years of experience in Germany,Number of vacation days,Company size,total compensation,Year,Month,Day,Hour,...,Main language at work_Italian,Main language at work_Russian,Main language at work_Spanish,Main language at work_other,Company type_Bank,Company type_Consulting / Agency,Company type_Media,Company type_Product,Company type_Startup,Company type_other
0,26.0,5.0,3.0,30.0,3.0,85000.0,2020,11,24,11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,26.0,7.0,4.0,28.0,4.0,85000.0,2020,11,24,11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,29.0,12.0,6.0,30.0,4.0,240000.0,2020,11,24,11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,28.0,4.0,1.0,24.0,3.0,59000.0,2020,11,24,11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,37.0,17.0,6.0,29.0,4.0,67000.0,2020,11,24,11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,32.0,5.0,1.0,30.0,2.0,81000.0,2020,11,24,11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6,37.0,6.0,0.4,24.0,2.0,62000.0,2020,11,24,11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7,24.0,5.0,1.0,27.0,5.0,70000.0,2020,11,24,11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8,29.0,8.0,2.0,28.0,4.0,61000.0,2020,11,24,11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,35.0,15.0,3.0,30.0,4.0,100000.0,2020,11,24,11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


##### Put X and Y into an array then ran my train test split

In [807]:
# df = df.dropna(subset=["total compensation"])
y = np.array(df["total compensation"]).reshape(-1,1)
X = np.array(df.drop(columns={"total compensation"}))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X.shape)
print(y.shape)

(1253, 65)
(1253, 1)


##### My base model grid search

In [808]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the imputer on the training set
imputer = SimpleImputer()
X_train = imputer.fit_transform(X_train)


# Impute the missing values in the test set
X_test = imputer.transform(X_test)


# Train the model
model = ExtraTreesRegressor()
model.fit(X_train, y_train.ravel())

# Evaluate the model on the test set
score = model.score(X_test, y_test.ravel())
print('Test set R^2:', score)

# Grid search
param_grid = {'n_estimators': [10, 20, 30],
              'max_depth': [2, 4, 6, 8],
              'min_samples_leaf': [1, 2, 4],
              'min_samples_split': [2, 4, 6, 8]}

grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(X_train, y_train.ravel())

# Print the best parameters and the best score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)

Test set R^2: -1441444548.3927484
Best parameters: {'max_depth': 2, 'min_samples_leaf': 4, 'min_samples_split': 6, 'n_estimators': 30}
Best cross-validation score: -606688710.8941066


##### Base Model

In [809]:
# Train the model with the best parameters
model = ExtraTreesRegressor(max_depth=2, min_samples_leaf=4, n_estimators=10, min_samples_split=6)
model.fit(X_train, y_train.ravel())

# Evaluate the new model on the test set
testscore = model.score(X_test, y_test.ravel())
trainscore= model.score(X_train, y_train.ravel())

# Calculate RMSE
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print('Test set R2:', testscore)
print('Train set R2', trainscore)
print('Test set RMSE:', rmse)


Test set R2: -753711659.809458
Train set R2 0.16528449792912037
Test set RMSE: 2104007706.8471


##### Used variance threshold to cut down on columns that were not correlated to the target

In [810]:
# Perform feature selection
selector = VarianceThreshold(0.2)
X_train_high_variance = selector.fit_transform(X)
print(X.shape)
print(X_train_high_variance.shape)

# Convert the transformed data back to a dataframe
X_train_df = pd.DataFrame(X)
X_train_high_variance_df = pd.DataFrame(X_train_high_variance, columns=X_train_df.columns[selector.get_support()])

# Perform the same feature selection on the test data
X_test_high_variance = selector.transform(X_test)

columns = X_train_df.columns[selector.get_support()]
print(columns)

(1253, 65)
(1253, 15)
Int64Index([0, 1, 2, 3, 4, 6, 7, 8, 13, 23, 24, 28, 29, 36, 62], dtype='int64')


##### improved model with target variable transformation, parameter testing and feature selection

In [811]:
# Apply the Box-Cox transformation to the target variable
y_transformed, lambda_ = boxcox(y.ravel())

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train_high_variance, y_transformed, test_size=0.2, random_state=42)

# Fit the imputer on the training set
imputer = SimpleImputer()
X_train = imputer.fit_transform(X_train)


# Impute the missing values in the test set
X_test = imputer.transform(X_test)


# Train the model
model = ExtraTreesRegressor()
model.fit(X_train, y_train.ravel())

# Evaluate the model on the test set
score = model.score(X_test, y_test.ravel())
print('Test set R^2:', score)

# Grid search
param_grid = {'n_estimators': [10, 20, 30],
              'max_depth': [2, 4, 6, 8],
              'min_samples_leaf': [1, 2, 4],
              'min_samples_split': [2, 4, 6, 8]}
              # 'criterion': ['squared_error'],
              # 'n_jobs': [-1, 2]}
            #   'oob_score': [True, False]}
# param_grid = {'n_estimators': [10, 20, 30],
#               'max_depth': [2, 4, 6, 8],
#               'min_samples_leaf': [1, 2, 4],
#               'min_samples_split': [2, 4, 6, 8],
#               'bootstrap': [True, False]}
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(X_train, y_train.ravel())

# Print the best parameters and the best score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)

Test set R^2: 0.2414616166238318
Best parameters: {'max_depth': 6, 'min_samples_leaf': 2, 'min_samples_split': 6, 'n_estimators': 20}
Best cross-validation score: 0.2119075827049432


##### Improved model results

In [812]:
# Train the model with the best parameters
model = ExtraTreesRegressor(max_depth=6, min_samples_leaf=1, n_estimators=10, min_samples_split=4)
model.fit(X_train, y_train.ravel())

# Evaluate the new model on the test set
testscore = model.score(X_test, y_test.ravel())
trainscore= model.score(X_train, y_train.ravel())

# Calculate RMSE
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print('Test set R2:', testscore)
print('Train set R2', trainscore)
print('Test set RMSE:', rmse)

Test set R2: 0.2504554065581178
Train set R2 0.48633643547568184
Test set RMSE: 0.009206036625850367


##### Everything below is code I was testing to try and find the best model

In [813]:
# regressor = ExtraTreesRegressor(n_estimators=20, max_depth=8, min_samples_leaf=1, min_samples_split=2)
# imputer = SimpleImputer(add_indicator=True)
# transformer = TransformedTargetRegressor(regressor=regressor, transformer=imputer)

# model2 = Pipeline([
#     ('imputer', imputer),
#     ('transformer', transformer)
# ])

# model2.fit(X_train, y_train.ravel())

# testscore = model2.score(X_test, y_test.ravel())
# trainscore= model2.score(X_train, y_train.ravel())
# print('Test set R2:', testscore)
# print('Train set R2', trainscore)

In [814]:
# # Create pipeline with imputer and scaler
# pipeline = Pipeline([
#     ('imputer', SimpleImputer(strategy='median')),
#     ('scaler', StandardScaler()),
#     ('model', RandomForestRegressor())
# ])

# # Define grid search parameters
# grid = {
#     'model__n_estimators': [50, 100, 200],
#     'model__max_depth': [5, 10, 20],
#     'model__min_samples_split': [2, 5, 10],
#     'model__min_samples_leaf': [1, 2, 4]
# }

# # Create grid search object
# grid_search = GridSearchCV(pipeline, grid, cv=5)

# # Fit grid search to data
# grid_search.fit(X_train, y_train.ravel())

# # Print the best parameters and best score
# print("Best parameters:", grid_search.best_params_)
# print("Best score:", grid_search.best_score_)

In [815]:

# numeric_transformer = Pipeline( steps=[
#         ("imputer", SimpleImputer(strategy="mean")),
#         ("scaler", StandardScaler())
#     ])

# preprocessor = Pipeline( steps=[
#         ("num", numeric_transformer)
#     ])

In [816]:

# best=Pipeline(steps=[('pre', preprocessor), ('classifier', RandomForestRegressor(max_depth=20, min_samples_leaf=2, min_samples_split=2, n_estimators= 100))])

# best.fit(X_train, y_train)

# print("Training Accuracy:", best.score(X_train, y_train))
# print("Testing Accuracy:", best.score(X_test, y_test)) 
# print(best._final_estimator)

In [817]:
# # Create the ridge regression model
# ridge_reg = Ridge()

# # Create the pipeline to process the data and train the model
# pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('ridge_reg', ridge_reg)])

# # Fit the pipeline to the training data
# pipeline.fit(X_train, y_train)

# # Evaluate the model on the test data
# testscore = pipeline.score(X_test, y_test.ravel())
# trainscore= pipeline.score(X_train, y_train.ravel())
# print('Test set R2:', testscore)
# print('Train set R2', trainscore)

# Answers and Explainations
(Expand/modify as needed)

### Results
##### Without feature selection, my initial model had a train score of 0.16 and a test score of over -1000000000. RMSE was also way out of line as well. This indicates that the model was was not a very good fit for the data.
##### After performing feature selection, my model's train score improved to 0.48 and the test score improved to 0.27 with a RMSE of 0.009. This suggests that the feature selection process was effective in reducing overfitting and improving the generalization of the model to unseen data. The gap between the train score and test score also decreased, which is a good sign that the model is not overfitting the training data.
### Feature Selection Activities
##### The first step in the feature selection process was to perform a frequency-based selection, where the top k values for each column were selected based on their frequency. The remaining values in the column were then replaced with the value "other". The purpose of this step was to reduce the number of unique values in each column, which can help to improve the performance of the model.
##### The second step in the feature selection process was to perform a variance-based selection, where features with low variance were removed. I had 67 columns to start and after feature selection I was left with 17. The purpose of this step was to remove features that do not contain much information or do not vary much, as they are unlikely to be useful for the model.
### Hyperparameter Changes 
#####  GridSearchCV was used to find the best hyperparameters for the model. The GridSearchCV was performed on the training set and searched over a defined set of hyperparameters, including "n_estimators", "max_depth", "min_samples_leaf", and "min_samples_split". The best hyperparameters were then obtained and their best cross-validation score was printed. I also messed around with the "n_jobs", "max depth", and "bootstrap" parameters but found that adding them in made the grid search too complex and/or they were unneeded in obtaining a high accuracy model.